In [1]:
# Quantum
import pennylane as qml
# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# Numpy, Pandas
import numpy as np
import pandas as pd
# Layer
from kan import KAN
from RNN_block import RNN_block
# Data processing
# from fucntions_1028 import data_seq, train_seq
from fucntions import data_seq, train_seq
from sklearn.preprocessing import MinMaxScaler
# Quantum User-Def Classes
from utils import my_utils
from NQE_class import NQE
from NQE_train_class import NQE_Train


In [2]:
# CSV 파일 경로
feature_path = './data/pca/pca_train_data_Perth.csv'
label_path = './data/pca/pca_label_data_Perth.csv'

batch_size = 32


new_size = (len(pd.read_csv(feature_path).values) // batch_size) * batch_size
print(new_size)

# CSV 파일을 읽어 Tensor로 변환
feature_data = torch.tensor(pd.read_csv(feature_path).values[:], dtype=torch.float32)
label_data = torch.tensor(pd.read_csv(label_path).values[:], dtype=torch.float32)

# data_seq 객체 생성
data = data_seq(feature_data, label_data)

train_loader, test_loader = data.split_data(test_ratio = 0.2, batch_size=batch_size, seq_first = False, for_nqe = True, n_sequence=0, seed = 40)
rnn_train_loader, rnn_test_loader = data.split_data(test_ratio = 0.2, batch_size=batch_size, seq_first = False, for_nqe = True, n_sequence=5, seed = 40)

3168
dataset length : 3168
test_size : 608
train_size : 2560
dataset length : 3163
test_size : 608
train_size : 2555


In [3]:
for t, l in train_loader:
    print(t.shape)
    print(l.shape)
    break

torch.Size([32, 2, 4])
torch.Size([32, 2])


In [4]:
for t, l in rnn_train_loader:
    print(t.shape)
    print(l.shape)
    break

torch.Size([32, 2, 5, 4])
torch.Size([32, 2])


In [5]:
def criterion(pred, label):
    '''
        pred : inner product of two states
        label : label data
    '''
    # print('pred : ', pred.shape)
    # print('label :', label.shape)
    # print('lbl[:, 0][-1] :', label[:, 0][-1])
    # print('2nd term :', (label[:, 0] * label[:, 1] + 1).shape)
    # loss = torch.sum((pred - label[:]) ** 2) / len(pred)
    loss = torch.sum(((pred) - 0.5 * (label[:, 0] * label[:, 1] + 1)) ** 2 ) / len(pred)
    return loss

def accuarcy(pred, label):
    '''
        pred : inner product of two states
        label : label data
    '''
    acc = torch.sum((torch.round(pred) == torch.round(0.5 * (label[:, 0] * label[:, 1] + 1)))) / len(pred)
    return acc

### FC NQE

In [6]:
fcnqe = NQE(n_feature=4, mode='FC')
fcnqe_train = NQE_Train(fcnqe, criterion, train_loader, test_loader, [accuarcy])
trained_fcnqe = fcnqe_train.train(500, seq_first=True)

=====Epoch : 1 =====
 loss : 0.09484 metric0 : 0.84375

 train_loss : 0.09484 test_loss : 0.15535 train_metric : 0.84375 test_metric : 0.79605

=====Epoch : 2 =====
 loss : 0.11835 metric0 : 0.81250

 train_loss : 0.11835 test_loss : 0.15188 train_metric : 0.81250 test_metric : 0.79441

=====Epoch : 3 =====
 loss : 0.10309 metric0 : 0.87500

 train_loss : 0.10309 test_loss : 0.15048 train_metric : 0.87500 test_metric : 0.79770

=====Epoch : 4 =====
 loss : 0.12521 metric0 : 0.84375

 train_loss : 0.12521 test_loss : 0.14334 train_metric : 0.84375 test_metric : 0.80921

=====Epoch : 5 =====
 loss : 0.11084 metric0 : 0.87500

 train_loss : 0.11084 test_loss : 0.14950 train_metric : 0.87500 test_metric : 0.79112

=====Epoch : 6 =====
 loss : 0.12429 metric0 : 0.81250

 train_loss : 0.12429 test_loss : 0.14629 train_metric : 0.81250 test_metric : 0.79770

=====Epoch : 7 =====
 loss : 0.17851 metric0 : 0.78125

 train_loss : 0.17851 test_loss : 0.15058 train_metric : 0.78125 test_metric : 0

In [7]:
torch.save(trained_fcnqe.state_dict(), "./models/pca_FC_NQE_loss169.pth")

### KANQE

In [8]:
kanqe = NQE(n_feature=4, mode='KAN')
kanqe_train = NQE_Train(kanqe, criterion, train_loader, test_loader, [accuarcy])
trained_kanqe = kanqe_train.train(500, seq_first=True)

checkpoint directory created: ./model
saving model version 0.0
=====Epoch : 1 =====
 loss : 0.62202 metric0 : 0.28125

 train_loss : 0.62202 test_loss : 0.59552 train_metric : 0.28125 test_metric : 0.32730

=====Epoch : 2 =====
 loss : 0.70507 metric0 : 0.25000

 train_loss : 0.70507 test_loss : 0.59520 train_metric : 0.25000 test_metric : 0.32730

=====Epoch : 3 =====
 loss : 0.47348 metric0 : 0.46875

 train_loss : 0.47348 test_loss : 0.60730 train_metric : 0.46875 test_metric : 0.32895

=====Epoch : 4 =====
 loss : 0.62969 metric0 : 0.31250

 train_loss : 0.62969 test_loss : 0.58993 train_metric : 0.31250 test_metric : 0.33224

=====Epoch : 5 =====
 loss : 0.43554 metric0 : 0.53125

 train_loss : 0.43554 test_loss : 0.59000 train_metric : 0.53125 test_metric : 0.33388

=====Epoch : 6 =====
 loss : 0.71426 metric0 : 0.21875

 train_loss : 0.71426 test_loss : 0.59110 train_metric : 0.21875 test_metric : 0.33388

=====Epoch : 7 =====
 loss : 0.61452 metric0 : 0.31250

 train_loss : 0.6

In [9]:
# torch.save(trained_kanqe.state_dict(), "./models/pca_KAN_NQE_loss{loss}.pth")

### RNNQE

In [10]:
rnnqe = NQE(n_feature=4, mode='RNN', rnn_sequence=5)
rnnqe_train = NQE_Train(rnnqe, criterion, rnn_train_loader, rnn_test_loader, [accuarcy])
trained_rnnqe = rnnqe_train.train(500, seq_first=True)

=====Epoch : 1 =====
 loss : 0.17818 metric0 : 0.81481

 train_loss : 0.17818 test_loss : 0.20344 train_metric : 0.81481 test_metric : 0.74013

=====Epoch : 2 =====
 loss : 0.08048 metric0 : 0.88889

 train_loss : 0.08048 test_loss : 0.17104 train_metric : 0.88889 test_metric : 0.76316

=====Epoch : 3 =====
 loss : 0.14420 metric0 : 0.85185

 train_loss : 0.14420 test_loss : 0.15398 train_metric : 0.85185 test_metric : 0.78783

=====Epoch : 4 =====
 loss : 0.11538 metric0 : 0.85185

 train_loss : 0.11538 test_loss : 0.16264 train_metric : 0.85185 test_metric : 0.78783

=====Epoch : 5 =====
 loss : 0.12779 metric0 : 0.81481

 train_loss : 0.12779 test_loss : 0.17849 train_metric : 0.81481 test_metric : 0.75329

=====Epoch : 6 =====
 loss : 0.15799 metric0 : 0.77778

 train_loss : 0.15799 test_loss : 0.16072 train_metric : 0.77778 test_metric : 0.77467

=====Epoch : 7 =====
 loss : 0.21228 metric0 : 0.74074

 train_loss : 0.21228 test_loss : 0.17420 train_metric : 0.74074 test_metric : 0

In [11]:
torch.save(trained_rnnqe.state_dict(), "./models/pca_RNNQE_loss209.pth")